In [15]:
# tutorial: https://github.com/ashima0109/VGG-classification

In [16]:
# import packages
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from sklearn.model_selection import train_test_split
import random
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
import gradio as gr

In [17]:
# construct model
def build(width, height, depth):
    # initialize model, input shape, and channel dimension
    model = Sequential()
    inputShape = (height, width, depth)
    chanDim = -1  

    # CONV -> RELU -> POOL layer set
    model.add(Conv2D(32, (3, 3), padding = "same", input_shape = inputShape))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis = chanDim))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.4))

    # (CONV -> RELU) * 2 -> POOL layer set
    model.add(Conv2D(64, (3, 3), padding = "same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis = chanDim))
    model.add(Conv2D(64, (3, 3), padding = "same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis = chanDim))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.4))

    # (CONV -> RELU) * 3 -> POOL layer set
    model.add(Conv2D(128, (3, 3), padding = "same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis = chanDim))
    model.add(Conv2D(128, (3, 3), padding = "same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis = chanDim))
    model.add(Conv2D(128, (3, 3), padding = "same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis = chanDim))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.4))

    # FC -> RELU layer set
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.8))

    # softmax classifier
    model.add(Dense(11, kernel_regularizer = 'l2'))
    model.add(Activation("softmax"))

    # return constructed model
    return model

In [18]:
# preprocessing
def preprocessing():
    DIRECTORY = r'new_dataset'
    CATEGORIES = ['1', '2A', '2B', '2C', '3A', '3B', '3C', '4A', '4B', '4C', 'no_hair']
    ENCODINGS = {
        '1': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        '2A': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        '2B': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        '2C': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        '3A': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        '3B': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        '3C': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        '4A': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        '4B': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        '4C': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        'no_hair': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
    }

    data = []

    for category in CATEGORIES:
        folder = os.path.join(DIRECTORY, category)

        for img in os.listdir(folder):
            img_path = os.path.join(folder, img)
            if (".DS_Store" in img_path):
                continue
            img_arr = cv2.imread(img_path)
            img_arr = cv2.resize(img_arr, (224, 224))
            encoding = ENCODINGS.get(category)
            data.append([img_arr, encoding])

    random.shuffle(data)

    X = []
    Y = []

    for features, labels in data:
        X.append(features)
        Y.append(labels)

    X = np.array(X)
    Y = np.array(Y)

    return (X, Y)

In [19]:
# testing
def test(x_test, y_test, model):
    for i in range(len(x_test)):
        img = x_test[i]
        plt.imshow(img)
        plt.show()
        prediction = model.predict(img.reshape(-1, 224, 224, 3))
        print("Prediction = " + str(prediction))
        print("Correct label = " + str(y_test[i]))

In [20]:
# training and validating
def train_and_validate():
    tup = preprocessing()
    X = tup[0]
    Y = tup[1]
    
    x_remainder, x_test, y_remainder, y_test = train_test_split(X, Y, test_size = 0.1, random_state = 42)
    x_train, x_valid, y_train, y_valid = train_test_split(x_remainder, y_remainder, test_size = 0.1, random_state = 42)

    x_train = x_train / 255.0
    x_valid = x_valid / 255.0
    x_test = x_test / 255.0     
    
    model = build(224, 224, 3)
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    model.summary()
    model.fit(x_train, y_train, epochs = 20, batch_size = 20, validation_data = (x_valid, y_valid))

    test(x_test, y_test, model)

In [ ]:
# get hair prediction -- this part needs to be updated
def make_prediction(img_path, model):
    img_arr = cv2.imread(img_path)
    img_arr = cv2.resize(img_arr, (224, 224))
    prediction = model.predict(img_arr.reshape(-1, 224, 224, 3))

    # this might not be correct
    ENCODINGS = {
        '1': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        '2A': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        '2B': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        '2C': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        '3A': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        '3B': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        '3C': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        '4A': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        '4B': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        '4C': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        'no_hair': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
    }

    for encoding in ENCODINGS:
        if prediction == encoding:
            return encoding

In [ ]:
train_and_validate()